# Qualicum Beach Orthomosaic Processing with GCPs

This notebook processes drone imagery to create orthomosaics using Agisoft Metashape, comparing results with and without ground control points (GCPs).

## Workflow:
1. Load GCPs from KMZ file
2. Download drone imagery from S3 (all 12 cells)
3. Process orthomosaic WITHOUT GCPs
4. Process orthomosaic WITH GCPs
5. Compare both orthomosaics against ESRI and OpenStreetMap basemaps
6. Generate comprehensive quality report


## Setup and Imports


In [1]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import warnings
import logging
warnings.filterwarnings('ignore')

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

# Add package to path
package_dir = Path.cwd()
sys.path.insert(0, str(package_dir))

from qualicum_beach_gcp_analysis import (
    load_gcps_from_kmz,
    download_basemap,
    visualize_gcps_on_basemap,
    calculate_gcp_bbox,
    download_all_images_from_input_dir,
    export_to_metashape_csv,
    process_orthomosaic,
    PhotoMatchQuality,
    DepthMapQuality,
    compare_orthomosaic_to_basemap,
    generate_comparison_report,
    generate_markdown_report,
)

print("✓ Imports successful!")


✓ Imports successful!


## Step 1: Load Ground Control Points


In [2]:
# Path to the KMZ file
kmz_path = "/Users/mauriciohessflores/Documents/Code/Data/Qualicum Beach GCPs/Spexi_Survey_Points/Spexi_Drone_Survey/QualicumBeach_AOI.kmz"

# Load GCPs
gcps = load_gcps_from_kmz(kmz_path)

print(f"\n✓ Loaded {len(gcps)} ground control points")

# Display all GCPs
if gcps:
    print("\nGCPs:")
    for i, gcp in enumerate(gcps):
        print(f"  {i+1:2d}. {gcp.get('id', 'Unknown'):20s}: ({gcp['lat']:.6f}, {gcp['lon']:.6f}, z={gcp.get('z', 0):.2f})")
else:
    print("\n⚠️  No GCPs found!")


Loading GCPs from: /Users/mauriciohessflores/Documents/Code/Data/Qualicum Beach GCPs/Spexi_Survey_Points/Spexi_Drone_Survey/QualicumBeach_AOI.kmz
Found 1 KML file(s) in KMZ
Attempting to fix namespace issues...
✓ Fixed namespace issues in KML file
Found 12 placemarks in KMZ file (namespace: http://www.opengis.net/kml/2.2)
Successfully parsed 12 GCPs from KMZ file

✓ Loaded 12 ground control points

GCPs:
   1. 8928d89ac03ffff     : (49.352544, -124.407904, z=0.00)
   2. 8928d89ac0bffff     : (49.354342, -124.404136, z=0.00)
   3. 8928d89ac1bffff     : (49.351585, -124.403857, z=0.00)
   4. 8928d89ac43ffff     : (49.355182, -124.396319, z=0.00)
   5. 8928d89ac47ffff     : (49.356141, -124.400367, z=0.00)
   6. 8928d89ac53ffff     : (49.352425, -124.396040, z=0.00)
   7. 8928d89ac57ffff     : (49.353384, -124.400088, z=0.00)
   8. 8928d89ac5bffff     : (49.354224, -124.392271, z=0.00)
   9. 8928d89ac73ffff     : (49.357100, -124.404415, z=0.00)
  10. 8928d89acc7ffff     : (49.348828, -12

## Step 2: Calculate Bounding Box and Download Reference Basemaps


In [3]:
# Create output directory
output_dir = Path("outputs")
output_dir.mkdir(exist_ok=True)

# Calculate bounding box
bbox = calculate_gcp_bbox(gcps, padding=0.01)
min_lat, min_lon, max_lat, max_lon = bbox

print(f"Bounding box:")
print(f"  Latitude: {min_lat:.6f} to {max_lat:.6f}")
print(f"  Longitude: {min_lon:.6f} to {max_lon:.6f}")

# Download ESRI World Imagery basemap (for comparison)
basemap_esri_path = str(output_dir / "qualicum_beach_basemap_esri.tif")
print("\nDownloading ESRI World Imagery basemap...")
basemap_esri_path = download_basemap(
    bbox=bbox,
    output_path=basemap_esri_path,
    source="esri_world_imagery",
    zoom=None
)
print(f"✓ ESRI basemap saved to: {basemap_esri_path}")

# Download OpenStreetMap basemap (for comparison)
basemap_osm_path = str(output_dir / "qualicum_beach_basemap_osm.tif")
print("\nDownloading OpenStreetMap basemap...")
basemap_osm_path = download_basemap(
    bbox=bbox,
    output_path=basemap_osm_path,
    source="openstreetmap",
    zoom=None
)
print(f"✓ OpenStreetMap basemap saved to: {basemap_osm_path}")


Bounding box:
  Latitude: 49.338828 to 49.367100
  Longitude: -124.417904 to -124.382271

Tile range: X [1264, 1265], Y [2800, 2801]
Basemap saved to outputs/qualicum_beach_basemap_esri.tif
✓ ESRI basemap saved to: outputs/qualicum_beach_basemap_esri.tif

Tile range: X [1264, 1265], Y [2800, 2801]
Basemap saved to outputs/qualicum_beach_basemap_osm.tif
✓ OpenStreetMap basemap saved to: outputs/qualicum_beach_basemap_osm.tif


## Step 3: Download Drone Imagery from S3


In [4]:
# Setup paths
input_dir = Path("input")
photos_dir = Path("input/images")

# Download all images from input manifest files
print("Downloading images from S3...")
print("=" * 60)
download_stats = download_all_images_from_input_dir(
    input_dir=input_dir,
    photos_dir=photos_dir,
    skip_existing=True  # Don't re-download if images already exist
)
print("=" * 60)
print("✓ Image download complete")


2025-11-25 13:36:22,200 - qualicum_beach_gcp_analysis.s3_downloader - INFO - Found 12 manifest files
2025-11-25 13:36:22,358 - qualicum_beach_gcp_analysis.s3_downloader - INFO - Processing manifest: input-file_172543.txt
2025-11-25 13:36:22,359 - qualicum_beach_gcp_analysis.s3_downloader - INFO -   Bucket: spexi-data-domain-assets-production-ca-central-1
2025-11-25 13:36:22,359 - qualicum_beach_gcp_analysis.s3_downloader - INFO -   S3 prefix: standardized-images/8928d89ac53ffff/172543/
2025-11-25 13:36:22,359 - qualicum_beach_gcp_analysis.s3_downloader - INFO -   Total images: 152
2025-11-25 13:36:22,377 - botocore.tokens - INFO - Loading cached SSO token for spexi


2025-11-25 13:36:27,458 - qualicum_beach_gcp_analysis.s3_downloader - INFO -   Downloaded 10/152 images...
2025-11-25 13:36:31,787 - qualicum_beach_gcp_analysis.s3_downloader - INFO -   Downloaded 20/152 images...
2025-11-25 13:36:35,733 - qualicum_beach_gcp_analysis.s3_downloader - INFO -   Downloaded 30/152 images...
2025-11-25 13:36:40,363 - qualicum_beach_gcp_analysis.s3_downloader - INFO -   Downloaded 40/152 images...
2025-11-25 13:36:44,594 - qualicum_beach_gcp_analysis.s3_downloader - INFO -   Downloaded 50/152 images...
2025-11-25 13:36:48,644 - qualicum_beach_gcp_analysis.s3_downloader - INFO -   Downloaded 60/152 images...
2025-11-25 13:36:53,876 - qualicum_beach_gcp_analysis.s3_downloader - INFO -   Downloaded 70/152 images...
2025-11-25 13:36:58,886 - qualicum_beach_gcp_analysis.s3_downloader - INFO -   Downloaded 80/152 images...
2025-11-25 13:37:03,928 - qualicum_beach_gcp_analysis.s3_downloader - INFO -   Downloaded 90/152 images...
2025-11-25 13:37:09,746 - qualicum_be

✓ Image download complete


## Step 4: Export GCPs for MetaShape


In [5]:
# Export GCPs to MetaShape CSV format
gcp_csv_path = output_dir / "gcps_metashape.csv"
export_to_metashape_csv(gcps, str(gcp_csv_path))
print(f"✓ GCPs exported to: {gcp_csv_path}")


Exported 12 GCPs to MetaShape CSV: outputs/gcps_metashape.csv
✓ GCPs exported to: outputs/gcps_metashape.csv


## Step 5: Process Orthomosaic WITHOUT GCPs


In [ ]:
# Setup paths for processing
intermediate_dir = output_dir / "intermediate"
ortho_output_dir = output_dir / "orthomosaics"

# Process orthomosaic WITHOUT GCPs
# Note: clean_intermediate_files=False will reuse existing processing steps
# Set to True to start fresh and delete previous work
print("=" * 60)
print("Processing orthomosaic WITHOUT GCPs...")
print("=" * 60)

project_path_no_gcps = intermediate_dir / "orthomosaic_no_gcps.psx"

stats_no_gcps = process_orthomosaic(
    photos_dir=photos_dir,
    output_path=ortho_output_dir,
    project_path=project_path_no_gcps,
    product_id="orthomosaic_no_gcps",
    clean_intermediate_files=False,  # Reuse existing processing if available
    photo_match_quality=PhotoMatchQuality.MediumQuality,
    depth_map_quality=DepthMapQuality.MediumQuality,
    tiepoint_limit=10000,
    use_gcps=False
)

print("\n✓ Orthomosaic processing (without GCPs) complete!")
print(f"  Number of photos: {stats_no_gcps['num_photos']}")
print(f"\n📁 Output Files:")
ortho_path_no_gcps = Path(stats_no_gcps['ortho_path'])
if ortho_path_no_gcps.exists():
    file_size_mb = ortho_path_no_gcps.stat().st_size / (1024 * 1024)
    print(f"  ✓ Orthomosaic GeoTIFF: {ortho_path_no_gcps.absolute()}")
    print(f"    Size: {file_size_mb:.2f} MB")
else:
    print(f"  ✗ Orthomosaic GeoTIFF NOT FOUND at: {ortho_path_no_gcps.absolute()}")
    print(f"    Expected location: {ortho_path_no_gcps}")
    print(f"    Output directory exists: {ortho_path_no_gcps.parent.exists()}")
if 'log_file_path' in stats_no_gcps:
    print(f"  📝 Log file: {stats_no_gcps['log_file_path']}")


2025-11-25 13:53:18,710 - qualicum_beach_gcp_analysis.metashape_processor - INFO - 🧹 Cleaning up previous processing files...
2025-11-25 13:53:18,712 - qualicum_beach_gcp_analysis.metashape_processor - INFO - 🚀 Initializing Metashape project...


Processing orthomosaic WITHOUT GCPs...
SaveProject: path = outputs/intermediate/orthomosaic_no_gcps.psx
saved project in 0.005374 sec
LoadProject: path = outputs/intermediate/orthomosaic_no_gcps.psx


2025-11-25 13:53:19,123 - qualicum_beach_gcp_analysis.metashape_processor - INFO - Adding photos from: input/images
2025-11-25 13:53:19,132 - qualicum_beach_gcp_analysis.metashape_processor - INFO - Found 1841 images


loaded project in 0.002411 sec
AddPhotos
SaveProject: path = /Users/mauriciohessflores/Documents/Code/MyCode/research-qualicum_beach_gcp_analysis/outputs/intermediate/orthomosaic_no_gcps.psx
saved project in 0.14287 sec


2025-11-25 13:53:20,765 - qualicum_beach_gcp_analysis.metashape_processor - INFO - Matching photos...


MatchPhotos: accuracy = Medium, preselection = generic, reference, keypoint limit = 40000, keypoint limit per mpx = 1000, tiepoint limit = 10000, apply masks = 0, filter tie points = 1, filter stationary points = 1, guided matching = 0
saved matching data in 0.000251 sec
scheduled 93 keypoint detection groups
saved keypoint partition in 0.000172 sec
groups: 2000 2000 2000 2000 2000 2000 2000 2000 2000 2000 2000 2000 2000 2000 2000 2000 2000 2000 2000 2000 2000 2000 2000 1994
4220 of 1841 used (229.223%)
scheduled 24 keypoint matching groups
saved matching partition in 0.000583 sec
loaded keypoint partition in 4.6e-05 sec
loaded matching data in 1.5e-05 sec
Found 1 GPUs in 0.046789 sec (OpenCL: 0.046778 sec)
Using device: Apple M4 Pro, 16 compute units, 36864 MB global memory, OpenCL 1.2
  driver version: 1.2 1.0, platform version: OpenCL 1.2 (Jul 20 2025 19:29:12)
  max work group size 256
  max work item sizes [256, 256, 256]
  max mem alloc size 6912 MB
Building OpenCL kernels for Ap

## Step 6: Process Orthomosaic WITH GCPs


In [ ]:
# Process orthomosaic WITH GCPs
# Note: clean_intermediate_files=False will reuse existing processing steps
# Set to True to start fresh and delete previous work
print("=" * 60)
print("Processing orthomosaic WITH GCPs...")
print("=" * 60)

project_path_with_gcps = intermediate_dir / "orthomosaic_with_gcps.psx"

stats_with_gcps = process_orthomosaic(
    photos_dir=photos_dir,
    output_path=ortho_output_dir,
    project_path=project_path_with_gcps,
    gcp_file=gcp_csv_path,
    product_id="orthomosaic_with_gcps",
    clean_intermediate_files=False,  # Reuse existing processing if available
    photo_match_quality=PhotoMatchQuality.MediumQuality,
    depth_map_quality=DepthMapQuality.MediumQuality,
    tiepoint_limit=10000,
    use_gcps=True
)

print("\n✓ Orthomosaic processing (with GCPs) complete!")
print(f"  Number of photos: {stats_with_gcps['num_photos']}")
print(f"  Number of markers: {stats_with_gcps.get('num_markers', 0)}")
print(f"\n📁 Output Files:")
ortho_path_with_gcps = Path(stats_with_gcps['ortho_path'])
if ortho_path_with_gcps.exists():
    file_size_mb = ortho_path_with_gcps.stat().st_size / (1024 * 1024)
    print(f"  ✓ Orthomosaic GeoTIFF: {ortho_path_with_gcps.absolute()}")
    print(f"    Size: {file_size_mb:.2f} MB")
else:
    print(f"  ✗ Orthomosaic GeoTIFF NOT FOUND at: {ortho_path_with_gcps.absolute()}")
    print(f"    Expected location: {ortho_path_with_gcps}")
    print(f"    Output directory exists: {ortho_path_with_gcps.parent.exists()}")
if 'log_file_path' in stats_with_gcps:
    print(f"  📝 Log file: {stats_with_gcps['log_file_path']}")


## Step 7: Compare Orthomosaics to Reference Basemaps


In [ ]:
# Compare against ESRI basemap
print("=" * 60)
print("Comparing orthomosaics to ESRI World Imagery basemap...")
print("=" * 60)

comparison_dir = output_dir / "comparisons"

# Compare without GCPs
print("\nComparing orthomosaic (without GCPs) to ESRI basemap...")
metrics_no_gcps_esri = compare_orthomosaic_to_basemap(
    ortho_path=Path(stats_no_gcps['ortho_path']),
    basemap_path=Path(basemap_esri_path),
    output_dir=comparison_dir
)

# Compare with GCPs
print("\nComparing orthomosaic (with GCPs) to ESRI basemap...")
metrics_with_gcps_esri = compare_orthomosaic_to_basemap(
    ortho_path=Path(stats_with_gcps['ortho_path']),
    basemap_path=Path(basemap_esri_path),
    output_dir=comparison_dir
)

print("\n✓ ESRI comparison complete!")


In [ ]:
# Compare against OpenStreetMap basemap
print("=" * 60)
print("Comparing orthomosaics to OpenStreetMap basemap...")
print("=" * 60)

# Compare without GCPs
print("\nComparing orthomosaic (without GCPs) to OpenStreetMap basemap...")
metrics_no_gcps_osm = compare_orthomosaic_to_basemap(
    ortho_path=Path(stats_no_gcps['ortho_path']),
    basemap_path=Path(basemap_osm_path),
    output_dir=comparison_dir
)

# Compare with GCPs
print("\nComparing orthomosaic (with GCPs) to OpenStreetMap basemap...")
metrics_with_gcps_osm = compare_orthomosaic_to_basemap(
    ortho_path=Path(stats_with_gcps['ortho_path']),
    basemap_path=Path(basemap_osm_path),
    output_dir=comparison_dir
)

print("\n✓ OpenStreetMap comparison complete!")


## Step 8: Generate Quality Reports


In [ ]:
# Generate report for ESRI comparison
print("=" * 60)
print("Generating quality reports...")
print("=" * 60)

# ESRI report
report_json_esri = output_dir / "quality_report_esri.json"
report_md_esri = output_dir / "quality_report_esri.md"

generate_comparison_report(
    metrics_with_gcps=metrics_with_gcps_esri,
    metrics_without_gcps=metrics_no_gcps_esri,
    output_path=report_json_esri,
    basemap_source="ESRI World Imagery"
)

generate_markdown_report(
    json_report_path=report_json_esri,
    output_path=report_md_esri
)

print(f"\n✓ ESRI report saved:")
print(f"  JSON: {report_json_esri}")
print(f"  Markdown: {report_md_esri}")

# OpenStreetMap report
report_json_osm = output_dir / "quality_report_osm.json"
report_md_osm = output_dir / "quality_report_osm.md"

generate_comparison_report(
    metrics_with_gcps=metrics_with_gcps_osm,
    metrics_without_gcps=metrics_no_gcps_osm,
    output_path=report_json_osm,
    basemap_source="OpenStreetMap"
)

generate_markdown_report(
    json_report_path=report_json_osm,
    output_path=report_md_osm
)

print(f"\n✓ OpenStreetMap report saved:")
print(f"  JSON: {report_json_osm}")
print(f"  Markdown: {report_md_osm}")


## Step 9: Display Report Summary


In [ ]:
# Display summary from ESRI report
import json

print("=" * 60)
print("QUALITY COMPARISON SUMMARY (ESRI World Imagery)")
print("=" * 60)

with open(report_json_esri, 'r') as f:
    report_esri = json.load(f)

comparison = report_esri.get('comparison', {})

if comparison.get('rmse_improvement'):
    rmse = comparison['rmse_improvement']
    print(f"\nRMSE Improvement: {rmse['percentage']:+.2f}%")
    print(f"  Without GCPs: {rmse['without_gcps']:.4f}")
    print(f"  With GCPs:    {rmse['with_gcps']:.4f}")

if comparison.get('mae_improvement'):
    mae = comparison['mae_improvement']
    print(f"\nMAE Improvement: {mae['percentage']:+.2f}%")
    print(f"  Without GCPs: {mae['without_gcps']:.4f}")
    print(f"  With GCPs:    {mae['with_gcps']:.4f}")

if comparison.get('similarity_improvement'):
    sim = comparison['similarity_improvement']
    print(f"\nSimilarity Improvement: {sim['percentage']:+.2f}%")
    print(f"  Without GCPs: {sim['without_gcps']:.4f}")
    print(f"  With GCPs:    {sim['with_gcps']:.4f}")

if comparison.get('seamline_reduction'):
    seam = comparison['seamline_reduction']
    print(f"\nSeamline Reduction: {seam['percentage']:+.2f}%")
    print(f"  Without GCPs: {seam['without_gcps']:.2f}%")
    print(f"  With GCPs:    {seam['with_gcps']:.2f}%")

print("\n" + "=" * 60)
print(f"Full reports available at:")
print(f"  {report_md_esri}")
print(f"  {report_md_osm}")
print("=" * 60)
